In [1]:
from pymongo import MongoClient

db = MongoClient().scraper_meta

In [53]:
c = db.samples.aggregate([{'$match': {'organism': 'Homo sapiens'}},
        {'$group':{
                '_id': '$characteristics.tissue',
                'count': {'$sum': 1}
            }}
    , {
        '$match': {'count': {'$gt': 1}}
    }, {
        '$sort': {'count': -1}
    }])

In [54]:
import pandas as pd

tissues_raw = pd.DataFrame(list(c))

In [55]:
tissues_raw.to_pickle('../data/age-biomarkers/geo-characterisitcs-tissues.pd3.pickle')
# tissues_raw = pd.read_pickle('../data/age-biomarkers/geo-characterisitcs-tissues.pd3.pickle')

In [56]:
tissues_raw['tissue'] = tissues_raw['_id'].str.lower()
tissues = pd.DataFrame(tissues_raw.groupby('tissue')['count'].sum()).sort(columns='count', ascending=False)

In [57]:
import qgrid
qgrid.show_grid(tissues)

In [58]:
qgrid.show_grid(tissues_raw[tissues_raw['tissue'] == 'liver'])

In [59]:
c = db.samples.aggregate([
                {'$match':{
                'characteristics.tissue': {'$in': ['liver', 'Liver', 'LIVER']},
                'organism': 'Homo sapiens'
            }}, {'$group':{
                '_id': '$platform',
                'count': {'$sum': 1}
            }}, {'$sort':{
                'count': -1
            }}])

In [60]:
liver_platforms = pd.DataFrame(list(c))
qgrid.show_grid(liver_platforms)

In [62]:
db.samples.count({
        'organism': 'Homo sapiens',
        'characteristics.Age': {'$exists': 1},
    })

8373